3. Processamento/tratamento de dados

Iniciamos o processamento dos dados obtendo os datasets de `delivery` e de `gas_prices` do Kaggle, por meio da biblioteca `kagglehub` em Python. Neste momento, definem-se também algumas funções de leitura dos arquivos (.CSV e .TSV) e de filtragem de datas, a ser usada posteriormente.

In [74]:
import kagglehub
import unicodedata
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup
from unidecode import unidecode

# Importa os datasets:
path_delivery = kagglehub.dataset_download("nosbielcs/brazilian-delivery-center")
path_gas_prices = kagglehub.dataset_download("matheusfreitag/gas-prices-in-brazil")

# Define função de leitura dos datasets:
def load_df(file_path):
    sep = "\t" if file_path.endswith(".tsv") else ","
    try:
        return pd.read_csv(file_path, sep=sep)
    except UnicodeDecodeError:
        return pd.read_csv(file_path, sep=sep, encoding="ISO-8859-1")

# Define função para encontrar a faixa de data correspondente para cada linha em 'delivery_orders':
def find_date_range(delivery_row, gas_df):
    # Filtra 'gas_prices_records' com base no estado e nas datas
    state_prices = gas_df[gas_df['state'] == delivery_row['state']]
    match = state_prices[
        (delivery_row['order_date'] >= state_prices['start_date']) &
        (delivery_row['order_date'] <= state_prices['end_date'])
    ]
    return match['date_range_id'].values[0] if not match.empty else None

In [75]:
# Define todos os 6 dataframes relevantes:
delivery_hubs = load_df(os.path.join(path_delivery, "hubs.csv"))
delivery_orders = load_df(os.path.join(path_delivery, "orders.csv"))
delivery_stores = load_df(os.path.join(path_delivery, "stores.csv"))
delivery_drivers = load_df(os.path.join(path_delivery, "drivers.csv"))
delivery_deliveries = load_df(os.path.join(path_delivery, "deliveries.csv"))
gas_prices_records = load_df(os.path.join(path_gas_prices, "2004-2021.tsv"))

Na definição dos dataframes importados das duas bases de dados, resolvemos omitir duas das 7 tabelas de Delivery Center: `delivery_channels` e `delivery_payments`. A omissão da `delivery_channels` se deu ao verificar que ela incluía apenas dados sobre o canal de origem dos pedidos — isto é, se o cliente efetuou a compra de um marketplace ou de um canal próprio da loja — pois essas informações não seriam relevantes para o objetivo geral da análise. Similarmente, também não cabe à análise qualquer dado de pagamento registrado na `delivery_payments`, pois não são dados diretamente relacionáveis ao fluxo de entregas e ao valor do combustível.

In [76]:
# Define delivery_channels e confere o seu conteúdo:
delivery_channels = load_df(os.path.join(path_delivery, "channels.csv"))
df = delivery_channels

print(f"-----------------------------------")
print(f"Analisando `delivery_channels`...")
print(f"-----------------------------------")
print("Tamanho:", df.shape,"\n")
print("Tipos de dados:\n", df.dtypes,"\n")
print("Lacunas:\n", df.isnull().sum(),"\n")
print("Sample Data:")
df.head()

-----------------------------------
Analisando `delivery_channels`...
-----------------------------------
Tamanho: (40, 3) 

Tipos de dados:
 channel_id       int64
channel_name    object
channel_type    object
dtype: object 

Lacunas:
 channel_id      0
channel_name    0
channel_type    0
dtype: int64 

Sample Data:


,channel_id,channel_name,channel_type
0,1,OTHER PLACE,OWN CHANNEL
1,2,PHONE PLACE,OWN CHANNEL
2,3,WHATS PLACE,OWN CHANNEL
3,4,FACE PLACE,OWN CHANNEL
4,5,FOOD PLACE,MARKETPLACE


In [77]:
# Define delivery_payments e confere o seu conteúdo:
delivery_payments = load_df(os.path.join(path_delivery, "payments.csv"))
df = delivery_payments

print(f"-----------------------------------")
print(f"Analisando 'delivery_payments'...")
print(f"-----------------------------------")
print("Tamanho:", df.shape,"\n")
print("Tipos de dados:\n", df.dtypes,"\n")
print("Lacunas:\n", df.isnull().sum(),"\n")
print("Sample Data:")
df.head()

-----------------------------------
Analisando 'delivery_payments'...
-----------------------------------
Tamanho: (400834, 6) 

Tipos de dados:
 payment_id            int64
payment_order_id      int64
payment_amount      float64
payment_fee         float64
payment_method       object
payment_status       object
dtype: object 

Lacunas:
 payment_id            0
payment_order_id      0
payment_amount        0
payment_fee         175
payment_method        0
payment_status        0
dtype: int64 

Sample Data:


,payment_id,payment_order_id,payment_amount,payment_fee,payment_method,payment_status
0,4427917,68410055,118.44,0.00,VOUCHER,PAID
1,4427918,68410055,394.81,7.90,ONLINE,PAID
2,4427941,68412721,206.95,5.59,ONLINE,PAID
3,4427948,68413340,58.80,1.59,ONLINE,PAID
4,4427955,68414018,45.80,0.92,ONLINE,PAID


In [78]:
# Define delivery_drivers e confere o seu conteúdo:
df = delivery_drivers

print(f"-----------------------------------")
print(f"Analisando 'delivery_drivers'...")
print(f"-----------------------------------")
print("Tamanho:", df.shape,"\n")
print("Tipos de dados:\n", df.dtypes,"\n")
print("Lacunas:\n", df.isnull().sum(),"\n")
print("Sample Data:")
df.head()

-----------------------------------
Analisando 'delivery_drivers'...
-----------------------------------
Tamanho: (4824, 3) 

Tipos de dados:
 driver_id        int64
driver_modal    object
driver_type     object
dtype: object 

Lacunas:
 driver_id       0
driver_modal    0
driver_type     0
dtype: int64 

Sample Data:


,driver_id,driver_modal,driver_type
0,133,MOTOBOY,LOGISTIC OPERATOR
1,138,MOTOBOY,FREELANCE
2,140,MOTOBOY,FREELANCE
3,143,BIKER,FREELANCE
4,148,MOTOBOY,FREELANCE


Para as 5 tabelas restantes de Delivery Center, resolvemos unificá-las na tabela central `delivery_orders`. São utilizadas as colunas chave de cada tabela, conforme organograma da Figura 1.

In [79]:
# Une dataframes 'drivers' e 'deliveries' por meio do 'driver_id':
delivery_deliveries = pd.merge(
    delivery_deliveries,
    delivery_drivers,
    on="driver_id",
    how="left",
)

# Une dataframes 'deliveries' e 'orders' por meio do 'delivery_order_id':
delivery_orders = pd.merge(
    delivery_orders,
    delivery_deliveries,
    on="delivery_order_id",
    how="left",
)

# Une dataframes 'hubs' e 'stores' por meio do 'hub_id':
delivery_stores = pd.merge(
    delivery_stores,
    delivery_hubs,
    on="hub_id",
    how="left",
)

# Une dataframes 'stores' e 'orders' por meio do 'store_id':
delivery_orders = pd.merge(
    delivery_orders,
    delivery_stores,
    on="store_id",
    how="left",
)

print(delivery_orders.columns)
print(delivery_orders.head())


Index(['order_id', 'store_id', 'channel_id', 'payment_order_id',
       'delivery_order_id', 'order_status', 'order_amount',
       'order_delivery_fee', 'order_delivery_cost', 'order_created_hour',
       'order_created_minute', 'order_created_day', 'order_created_month',
       'order_created_year', 'order_moment_created', 'order_moment_accepted',
       'order_moment_ready', 'order_moment_collected',
       'order_moment_in_expedition', 'order_moment_delivering',
       'order_moment_delivered', 'order_moment_finished',
       'order_metric_collected_time', 'order_metric_paused_time',
       'order_metric_production_time', 'order_metric_walking_time',
       'order_metric_expediton_speed_time', 'order_metric_transit_time',
       'order_metric_cycle_time', 'delivery_id', 'driver_id',
       'delivery_distance_meters', 'delivery_status', 'driver_modal',
       'driver_type', 'hub_id', 'store_name', 'store_segment',
       'store_plan_price', 'store_latitude', 'store_longitude', 'hub_

Após união das 5 tabelas, a limpeza desta é feita em algumas etapas:

1. Remover colunas que não têm relevância para a análise (que "sobraram" do merge da etapa anterior, ou se tratam de IDs que não serão mais utilizados em novos merges): `channel_id`, `payment_order_id`, `driver_type`, `store_name`, `store_latitude`, `store_longitude`, `store_plan_price`, `store_segment`, `hub_name`, `hub_latitude`, `hub_longitude`

2. Examinar todas as colunas de data para determinar quais devemos manter no dataset: `order_created_hour`, `order_created_minute`, `order_created_day`, `order_created_month`, `order_created_year`, `order_moment_created`, `order_moment_accepted`, `order_moment_ready`, `order_moment_collected`, `order_moment_in_expedition`, `order_moment_delivering`, `order_moment_delivered`, `order_moment_finished`

Nestes casos, as colunas com prefixo `order_moment` continham o datetime completo e tornam irrisórias as colunas de `hour`, `minute`, `day`, `month` e `year`. Já mesmo entre as colunas de `order_moment`, não precisamos de todas as informações oferecidas; basta que tenhamos a informação a nível de dia, dentre as quais julgamos que as duas colunas mais adequadas seriam `order_moment_created` e `order_moment_finished`. No entanto, não seria necessário manter ambas datas de criação e de finalização do pedido; entre as duas, resolvemos que faria mais sentido utilizar a data de criação do pedido, pois considera a data de aceitação por parte da loja e representa, portanto, a data de funcionamento desta. Desse modo, removemos todas as colunas de data, exceto `order_moment_created`.

3. Examinar as colunas de métrica de tempo para determinar quais devemos manter no dataset: `order_metric_collected_time`, `order_metric_paused_time`, `order_metric_production_time`, `order_metric_walking_time`, `order_metric_expediton_speed_time`, `order_metric_transit_time`, `order_metric_cycle_time`

Quanto às colunas de métrica, nalisamos os tipos de dados que estão sendo disponibilizados por cada uma, bem como validamos cada variável a respeito de lacunas. Neste momento, foi levantada a hipótese de que as métricas só estivessem presentes para os casos que `driver_id` não fosse nulo, pois são colunas originadas da tabela `delivery_drivers` — e estava correta. Dessa forma, removemos todas as linhas de `delivery_orders` para as quais `driver_id` estivesse em branco, e removemos as colunas de métrica que não são diretamente relacionadas à etapa de transporte feita pelo entregador: `order_metric_collected_time`, `order_metric_production_time`, `order_metric_walking_time` e `order_metric_cycle_time`.

In [80]:
# Passo 1 - Remove colunas irrelevantes:
cols_to_remove = [
    'channel_id', 'payment_order_id', 'driver_type', 
    'store_name', 'store_latitude', 'store_longitude', 'store_plan_price',
    'store_segment', 'hub_name', 'hub_latitude', 'hub_longitude'
]

print(delivery_orders.columns)
delivery_orders = delivery_orders.drop(columns=cols_to_remove)
print(delivery_orders.columns)

Index(['order_id', 'store_id', 'channel_id', 'payment_order_id',
       'delivery_order_id', 'order_status', 'order_amount',
       'order_delivery_fee', 'order_delivery_cost', 'order_created_hour',
       'order_created_minute', 'order_created_day', 'order_created_month',
       'order_created_year', 'order_moment_created', 'order_moment_accepted',
       'order_moment_ready', 'order_moment_collected',
       'order_moment_in_expedition', 'order_moment_delivering',
       'order_moment_delivered', 'order_moment_finished',
       'order_metric_collected_time', 'order_metric_paused_time',
       'order_metric_production_time', 'order_metric_walking_time',
       'order_metric_expediton_speed_time', 'order_metric_transit_time',
       'order_metric_cycle_time', 'delivery_id', 'driver_id',
       'delivery_distance_meters', 'delivery_status', 'driver_modal',
       'driver_type', 'hub_id', 'store_name', 'store_segment',
       'store_plan_price', 'store_latitude', 'store_longitude', 'hub_

In [81]:
# Passo 2.1 - Examina colunas de data e hora:
print(delivery_orders[['order_created_hour', 'order_created_minute', 'order_created_day', 
                       'order_created_month', 'order_created_year', 'order_moment_created', 
                       'order_moment_accepted', 'order_moment_ready', 'order_moment_collected', 
                       'order_moment_in_expedition', 'order_moment_delivering', 
                       'order_moment_delivered', 'order_moment_finished']].head())

   order_created_hour  order_created_minute  order_created_day  \
0                   0                     1                  1   
1                   0                     4                  1   
2                   0                    13                  1   
3                   0                    19                  1   
4                   0                    26                  1   

   order_created_month  order_created_year  order_moment_created  \
0                    1                2021  1/1/2021 12:01:36 AM   
1                    1                2021  1/1/2021 12:04:26 AM   
2                    1                2021  1/1/2021 12:13:07 AM   
3                    1                2021  1/1/2021 12:19:15 AM   
4                    1                2021  1/1/2021 12:26:25 AM   

  order_moment_accepted order_moment_ready order_moment_collected  \
0                   NaN                NaN                    NaN   
1                   NaN                NaN              

In [82]:
# Passo 2.2 - Remove colunas irrelevantes de data e hora:
cols_to_remove = [
    'order_created_hour', 'order_created_minute', 'order_created_day', 
    'order_created_month', 'order_created_year', 'order_moment_accepted', 
    'order_moment_ready', 'order_moment_collected', 'order_moment_in_expedition', 
    'order_moment_delivering', 'order_moment_delivered', 'order_moment_finished',
    'order_amount', 'order_delivery_fee', 'order_delivery_cost'
]

delivery_orders = delivery_orders.drop(columns=cols_to_remove)
print(delivery_orders.columns)

Index(['order_id', 'store_id', 'delivery_order_id', 'order_status',
       'order_moment_created', 'order_metric_collected_time',
       'order_metric_paused_time', 'order_metric_production_time',
       'order_metric_walking_time', 'order_metric_expediton_speed_time',
       'order_metric_transit_time', 'order_metric_cycle_time', 'delivery_id',
       'driver_id', 'delivery_distance_meters', 'delivery_status',
       'driver_modal', 'hub_id', 'hub_city', 'hub_state'],
      dtype='object')


In [83]:
# Passo 3.1 - Examina colunas de métrica:
delivery_orders_metric = delivery_orders[['order_id','driver_id','order_metric_collected_time', 'order_metric_paused_time',
                       'order_metric_production_time', 'order_metric_walking_time',
                       'order_metric_expediton_speed_time', 'order_metric_transit_time',
                       'order_metric_cycle_time']]
print(delivery_orders_metric.head())

# Passo 3.2 - Examina lacunas nas colunas de métrica:
empty_columns = delivery_orders_metric.loc[:, delivery_orders_metric.isna().all()].columns
print("Colunas completamente vazias:", empty_columns)

for column in delivery_orders_metric.columns:
    unique_values = delivery_orders_metric[column].unique()
    print(f"Valores únicos na coluna '{column}':", unique_values)

# Passo 3.3 - Confirma a suspeita de que os valores de métrica não sejam nulos para casos em que o 'driver_id' não for nulo:
delivery_orders_metric = delivery_orders[['driver_id','order_metric_expediton_speed_time', 
                                    'order_metric_paused_time', 
                                    'order_metric_transit_time']]

filtered_delivery_orders_metric = delivery_orders_metric[delivery_orders_metric['driver_id'].notnull()]
print(filtered_delivery_orders_metric.head())

# Passo 3.4 - Remove colunas irrelevantes de métrica:
cols_to_remove = [
    'order_metric_collected_time',
    'order_metric_production_time', 'order_metric_walking_time',
    'order_metric_cycle_time'
]

print(delivery_orders.columns)
delivery_orders = delivery_orders.drop(columns=cols_to_remove)
print(delivery_orders.columns)

   order_id  driver_id  order_metric_collected_time  order_metric_paused_time  \
0  68405119        NaN                          NaN                       NaN   
1  68405123        NaN                          NaN                       NaN   
2  68405206        NaN                          NaN                       NaN   
3  68405465        NaN                          NaN                       NaN   
4  68406064        NaN                          NaN                       NaN   

   order_metric_production_time  order_metric_walking_time  \
0                           NaN                        NaN   
1                           NaN                        NaN   
2                           NaN                        NaN   
3                           NaN                        NaN   
4                           NaN                        NaN   

   order_metric_expediton_speed_time  order_metric_transit_time  \
0                                NaN                        NaN   
1     

Conferindo a estrutura atual da tabela delivery_orders, verificamos se ainda há alguma coluna que pode ser filtrada e/ou omitida para simplificá-la.

* A coluna `order_status` tem os seguintes valores únicos: 'CANCELED', 'FINISHED'
* A coluna `delivery_status` tem os seguintes valores únicos: 'CANCELLED', 'DELIVERED', 'DELIVERING' e (VAZIO)
* A coluna `driver_modal` tem os seguintes valores únicos: 'BIKER', 'MOTOBOY', (VAZIO)

Não pretendemos analisar fatores que levem o cliente ou a loja a cancelarem um pedido. Logo, podemos filtrar `order_status` somente para 'FINISHED', e então remover a coluna. Também escolhemos dar importância somente aos pedidos que seguiram o ciclo completo de compra e entrega para o cliente, garantindo que a etapa de entrega foi finalizada; portanto, filtramos `delivery_status` por 'DELIVERED' e removemos a coluna. No caso de `driver_modal`, em se tratando de uma análise sobre preços de combustível, são relevantes apenas os casos de entregadores do tipo 'MOTOBOY', e então a coluna também pode ser removida.

In [84]:
# Verifica os valores únicos de 'order_status':
print("Valores únicos de order_status:", delivery_orders['order_status'].unique())

# Verifica os valores únicos de 'delivery_status':
print("Valores únicos de delivery_status:", delivery_orders['delivery_status'].unique())

# Verifica os valores únicos de 'driver_modal':
print("Valores únicos de delivery_status:", delivery_orders['driver_modal'].unique())

Valores únicos de order_status: ['CANCELED' 'FINISHED']
Valores únicos de delivery_status: [nan 'CANCELLED' 'DELIVERED' 'DELIVERING']
Valores únicos de delivery_status: [nan 'BIKER' 'MOTOBOY']


In [85]:
# Filtra o dataframe para manter apenas as linhas com 'order_status' = 'FINISHED':
delivery_orders = delivery_orders[delivery_orders['order_status'] == 'FINISHED']
delivery_orders = delivery_orders.drop(columns=['order_status'])

# Filtra o dataframe para manter apenas as linhas com 'order_status' = 'DELIVERED':
delivery_orders = delivery_orders[delivery_orders['delivery_status'] == 'DELIVERED']
delivery_orders = delivery_orders.drop(columns=['delivery_status'])

# Filtra o dataframe para manter apenas as linhas com 'driver_modal' = 'MOTOBOY':
delivery_orders = delivery_orders[delivery_orders['driver_modal'] == 'MOTOBOY']
delivery_orders = delivery_orders.drop(columns=['driver_modal'])

# Filtra o dataframe para manter apenas as linhas com 'driver_id' não nulo:
delivery_orders = delivery_orders[delivery_orders['driver_id'].notna()]
delivery_orders = delivery_orders.drop(columns=['driver_id'])

In [86]:
delivery_orders.head()

,order_id,store_id,delivery_order_id,order_moment_created,order_metric_paused_time,order_metric_expediton_speed_time,order_metric_transit_time,delivery_id,delivery_distance_meters,hub_id,hub_city,hub_state
17,68412721,631,68412721,1/1/2021 2:12:11 PM,10.22,11.05,83.30,2174663.0,5714.0,28,SÃO PAULO,SP
18,68413340,631,68413340,1/1/2021 2:14:51 PM,3.10,12.67,49.78,2174658.0,5199.0,28,SÃO PAULO,SP
19,68414018,3265,68414018,1/1/2021 2:17:31 PM,3.03,13.28,11.05,2174675.0,3746.0,37,SÃO PAULO,SP
21,68414512,631,68414512,1/1/2021 2:24:01 PM,39.75,39.85,23.82,2174735.0,5514.0,28,SÃO PAULO,SP
26,68415344,631,68415344,1/1/2021 2:33:32 PM,28.83,29.38,53.08,2174733.0,5385.0,28,SÃO PAULO,SP


In [87]:
# Remove linhas que estejam com a data vazia em 'order_moment_created':
delivery_orders = delivery_orders.dropna(subset=['order_moment_created'])

# Normaliza a data para que o horário seja definido para 00:00:00:
delivery_orders['order_moment_created'] = pd.to_datetime(delivery_orders['order_moment_created']).dt.normalize()

# Confere tipos de dados da tabela após conversões:
print(delivery_orders.dtypes)

C:\Users\clara\AppData\Local\Temp\ipykernel_1932\758155084.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  delivery_orders['order_moment_created'] = pd.to_datetime(delivery_orders['order_moment_created']).dt.normalize()


order_id                                      int64
store_id                                      int64
delivery_order_id                             int64
order_moment_created                 datetime64[ns]
order_metric_paused_time                    float64
order_metric_expediton_speed_time           float64
order_metric_transit_time                   float64
delivery_id                                 float64
delivery_distance_meters                    float64
hub_id                                        int64
hub_city                                     object
hub_state                                    object
dtype: object


In [88]:
# Remove IDs restantes indesejados: 'delivery_id', 'hub_id', 'store_id', 'order_id':
cols_to_remove = ['delivery_id', 'hub_id', 'store_id', 'order_id']

delivery_orders = delivery_orders.drop(columns=cols_to_remove)

In [89]:
# Renomeia colunas para tornar o dataframe mais intuitivo:
delivery_orders = delivery_orders.rename(columns={
    'order_moment_created': 'order_date',
    'hub_city': 'city',
    'hub_state': 'state',
    'order_metric_paused_time': 'paused_time_minutes',
    'order_metric_expediton_speed_time': 'speed_time_minutes',
    'order_metric_transit_time': 'transit_time_minutes'
})

# Conferindo o resultado
delivery_orders.head()

,delivery_order_id,order_date,paused_time_minutes,speed_time_minutes,transit_time_minutes,delivery_distance_meters,city,state
17,68412721,2021-01-01,10.22,11.05,83.30,5714.0,SÃO PAULO,SP
18,68413340,2021-01-01,3.10,12.67,49.78,5199.0,SÃO PAULO,SP
19,68414018,2021-01-01,3.03,13.28,11.05,3746.0,SÃO PAULO,SP
21,68414512,2021-01-01,39.75,39.85,23.82,5514.0,SÃO PAULO,SP
26,68415344,2021-01-01,28.83,29.38,53.08,5385.0,SÃO PAULO,SP


Já o processamento de Gas Prices é mais direto, já que o dataset continha apenas uma tabela, que chamamos de `gas_prices_records`.

In [90]:
gas_prices_records.head()

,DATA INICIAL,DATA FINAL,REGIÃO,ESTADO,PRODUTO,NÚMERO DE POSTOS PESQUISADOS,UNIDADE DE MEDIDA,PREÇO MÉDIO REVENDA,DESVIO PADRÃO REVENDA,PREÇO MÍNIMO REVENDA,PREÇO MÁXIMO REVENDA,MARGEM MÉDIA REVENDA,COEF DE VARIAÇÃO REVENDA,PREÇO MÉDIO DISTRIBUIÇÃO,DESVIO PADRÃO DISTRIBUIÇÃO,PREÇO MÍNIMO DISTRIBUIÇÃO,PREÇO MÁXIMO DISTRIBUIÇÃO,COEF DE VARIAÇÃO DISTRIBUIÇÃO
0,2004-05-09,2004-05-15,CENTRO OESTE,DISTRITO FEDERAL,ETANOL HIDRATADO,127,R$/l,1.288,0.016,1.19,1.350,0.463,0.012,0.825,0.11,0.4201,0.9666,0.133
1,2004-05-09,2004-05-15,CENTRO OESTE,GOIAS,ETANOL HIDRATADO,387,R$/l,1.162,0.114,0.89,1.449,0.399,0.098,0.763,0.088,0.5013,1.05,0.115
2,2004-05-09,2004-05-15,CENTRO OESTE,MATO GROSSO,ETANOL HIDRATADO,192,R$/l,1.389,0.097,1.18,1.760,0.419,0.070,0.97,0.095,0.5614,1.161,0.098
3,2004-05-09,2004-05-15,CENTRO OESTE,MATO GROSSO DO SUL,ETANOL HIDRATADO,162,R$/l,1.262,0.070,1.09,1.509,0.432,0.055,0.83,0.119,0.5991,1.22242,0.143
4,2004-05-09,2004-05-15,NORDESTE,ALAGOAS,ETANOL HIDRATADO,103,R$/l,1.181,0.078,1.05,1.400,0.24,0.066,0.941,0.077,0.7441,1.0317,0.082


Dentre as colunas do dataset, as únicas que julgamos relevante manter foram: `DATA INICIAL`, `DATA FINAL`, `REGIÃO`, `ESTADO`, `PRODUTO` e `PREÇO MÉDIO REVENDA`. Após limpeza das colunas, padronizamos a nomenclatura dos cabeçalhos das colunas restantes para seguirem o mesmo padrão existente em `delivery_orders`: nomes em inglês, todos em minúsculas, sem acentos e separados por underscore em vez de espaços.

In [91]:
# Mantém apenas as colunas relevantes de 'gas_prices_records':
gas_prices_records = gas_prices_records[['DATA INICIAL', 'DATA FINAL', 'ESTADO', 'PRODUTO', 'PREÇO MÉDIO REVENDA']]
gas_prices_records.head()

,DATA INICIAL,DATA FINAL,ESTADO,PRODUTO,PREÇO MÉDIO REVENDA
0,2004-05-09,2004-05-15,DISTRITO FEDERAL,ETANOL HIDRATADO,1.288
1,2004-05-09,2004-05-15,GOIAS,ETANOL HIDRATADO,1.162
2,2004-05-09,2004-05-15,MATO GROSSO,ETANOL HIDRATADO,1.389
3,2004-05-09,2004-05-15,MATO GROSSO DO SUL,ETANOL HIDRATADO,1.262
4,2004-05-09,2004-05-15,ALAGOAS,ETANOL HIDRATADO,1.181


In [92]:
# Renomeia colunas de 'gas_prices_records' para seguir o padrão de 'delivery_orders':
gas_prices_records = gas_prices_records.rename(columns={
    'DATA INICIAL': 'start_date',
    'DATA FINAL': 'end_date',
    'REGIÃO': 'region',
    'ESTADO': 'state_name',
    'PRODUTO': 'product',
    'PREÇO MÉDIO REVENDA': 'average_retail_price'
})

gas_prices_records.head()

,start_date,end_date,state_name,product,average_retail_price
0,2004-05-09,2004-05-15,DISTRITO FEDERAL,ETANOL HIDRATADO,1.288
1,2004-05-09,2004-05-15,GOIAS,ETANOL HIDRATADO,1.162
2,2004-05-09,2004-05-15,MATO GROSSO,ETANOL HIDRATADO,1.389
3,2004-05-09,2004-05-15,MATO GROSSO DO SUL,ETANOL HIDRATADO,1.262
4,2004-05-09,2004-05-15,ALAGOAS,ETANOL HIDRATADO,1.181


In [93]:
# Confere todos os tipos de combustível em 'product':
unique_products = gas_prices_records['product'].unique()
print(unique_products)

# Mantém somente os combustíveis adequados para motocicletas:
gas_prices_records = gas_prices_records[gas_prices_records['product'].isin(['ETANOL HIDRATADO', 'GASOLINA COMUM', 'GASOLINA ADITIVADA'])]

['ETANOL HIDRATADO' 'GASOLINA COMUM' 'GLP' 'GNV' 'ÓLEO DIESEL'
 'ÓLEO DIESEL S10' 'OLEO DIESEL' 'OLEO DIESEL S10' 'GASOLINA ADITIVADA']


Para união da tabela `gas_prices_records` com a `delivery_orders`, seria preciso padronizar o formato dos UFs na coluna `states`. Como a tabela de Gas Prices continha somente o nome dos estados por extenso, buscamos uma base confiável do IBGE e importamos a tabela por meio de webscraping com a biblioteca BeautifulSoup em Python. Após importação, a tabela exigiu que as colunas fossem repadronizadas para se assimilarem às outras tabelas, e também precisou de pequeno tratamento no registro do Distrito Federal (havia um asterisco indesejado).

In [94]:
# Obtém dados de página da web sobre os estados brasileiros:
url = "https://brasilescola.uol.com.br/brasil/estados-brasil.htm"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
table = soup.find('table', {'class': 'Table'})

state_names_list = []
states_list = []

# Itera sobre as linhas da tabela, a partir da segunda linha:
for row in table.find_all('tr')[1:]:
    cols = row.find_all('td')
    state_names_list.append(cols[0].text.strip())
    states_list.append(cols[2].text.strip())

# Define o dataframe 'states_df':
states_df = pd.DataFrame({
    'state_name': state_names_list,
    'state': states_list
})

# Faz ajustes à coluna 'state_name' para confronto com 'gas_prices_records':
states_df['state_name'] = states_df['state_name'].str.upper()
states_df['state_name'] = states_df['state_name'].str.replace('*', '', regex=False)
states_df['state_name'] = states_df['state_name'].apply(
    lambda text: ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    )
)

states_df


,state_name,state
0,ACRE,AC
1,ALAGOAS,AL
2,AMAPA,AP
3,AMAZONAS,AM
4,BAHIA,BA
5,CEARA,CE
6,DISTRITO FEDERAL,DF
7,ESPIRITO SANTO,ES
8,GOIAS,GO
9,MARANHAO,MA


In [95]:
# Une dataframes 'states_df' e 'gas_prices_records' por meio do 'state_name':
gas_prices_records = pd.merge(
    gas_prices_records,
    states_df,
    on="state_name",
    how="left",
)

In [96]:
gas_prices_records = gas_prices_records.drop(['state_name'], axis=1)
gas_prices_records.head()

,start_date,end_date,product,average_retail_price,state
0,2004-05-09,2004-05-15,ETANOL HIDRATADO,1.288,DF
1,2004-05-09,2004-05-15,ETANOL HIDRATADO,1.162,GO
2,2004-05-09,2004-05-15,ETANOL HIDRATADO,1.389,MT
3,2004-05-09,2004-05-15,ETANOL HIDRATADO,1.262,MS
4,2004-05-09,2004-05-15,ETANOL HIDRATADO,1.181,AL


A fim de, finalmente, unir as colunas de `gas_prices_records` a `delivery_orders`, precisamos determinar os períodos de sobreposição de datas dos dois datasets: ambos contêm dados plenos no período de 2021-01-01 a 2021-04-30 (que é também todo o período contido em `delivery_orders`).

In [97]:
# Checa o mínimo e máximo de data em 'gas_prices_records':
min_gas_price_date = gas_prices_records['start_date'].min()
max_gas_price_date = gas_prices_records['end_date'].max()
print(f"Intervalo de data em gas_prices_records: {min_gas_price_date} a {max_gas_price_date}")

# Checa o mínimo e máximo de data em 'delivery_orders':
min_delivery_date = delivery_orders['order_date'].min()
max_delivery_date = delivery_orders['order_date'].max()
print(f"Intervalo de data em delivery_orders: {min_delivery_date} a {max_delivery_date}")

Intervalo de data em gas_prices_records: 2004-05-09 a 2021-05-01
Intervalo de data em delivery_orders: 2021-01-01 00:00:00 a 2021-04-30 00:00:00


Neste momento, chegando mais próximo da etapa de análise, resolvemos manter somente o combustível mais comumente usado (GASOLINA COMUM), evitando uma possível futura ambiguidade das análises.

In [98]:
# Filtra 'gas_prices_records' para incluir dados somente a partir de 2021-01-01 e somente o combustível mais comum:
gas_prices_records = gas_prices_records[gas_prices_records['end_date'] >= '2021-01-01']
gas_prices_records = gas_prices_records[gas_prices_records['product']=='GASOLINA COMUM']

In [99]:
# Confere o conteúdo final em delivery_orders:
df = delivery_orders

print(f"-----------------------------------")
print(f"Analisando 'delivery_orders'...")
print(f"-----------------------------------")
print("Tamanho:", df.shape,"\n")
print("Tipos de dados:\n", df.dtypes,"\n")
print("Lacunas:\n", df.isnull().sum(),"\n")
print("Sample Data:")
df.head()

-----------------------------------
Analisando 'delivery_orders'...
-----------------------------------
Tamanho: (263930, 8) 

Tipos de dados:
 delivery_order_id                    int64
order_date                  datetime64[ns]
paused_time_minutes                float64
speed_time_minutes                 float64
transit_time_minutes               float64
delivery_distance_meters           float64
city                                object
state                               object
dtype: object 

Lacunas:
 delivery_order_id               0
order_date                      0
paused_time_minutes         40116
speed_time_minutes          10461
transit_time_minutes         3023
delivery_distance_meters       25
city                            0
state                           0
dtype: int64 

Sample Data:


,delivery_order_id,order_date,paused_time_minutes,speed_time_minutes,transit_time_minutes,delivery_distance_meters,city,state
17,68412721,2021-01-01,10.22,11.05,83.30,5714.0,SÃO PAULO,SP
18,68413340,2021-01-01,3.10,12.67,49.78,5199.0,SÃO PAULO,SP
19,68414018,2021-01-01,3.03,13.28,11.05,3746.0,SÃO PAULO,SP
21,68414512,2021-01-01,39.75,39.85,23.82,5514.0,SÃO PAULO,SP
26,68415344,2021-01-01,28.83,29.38,53.08,5385.0,SÃO PAULO,SP


E, a fim de facilitar a determinação de a qual faixa de datas pertence cada `delivery_order_id`, atribuímos números únicos a cada faixa de tempo (semana) na forma da coluna `date_range_id`. Após essa etapa, unimos os dois datasets por meio da coluna `state` e da nova `date_range_id`.

In [100]:
# Converte colunas de data para o formato 'datetime', caso não estejam:
delivery_orders['order_date'] = pd.to_datetime(delivery_orders['order_date'])
gas_prices_records['start_date'] = pd.to_datetime(gas_prices_records['start_date'])
gas_prices_records['end_date'] = pd.to_datetime(gas_prices_records['end_date'])

# Enumera faixas de tempo em 'gas_prices_records' para facilitar a mesclagem com 'delivery_orders':
gas_prices_records['date_range_id'] = gas_prices_records.groupby(['state']).cumcount() + 1
gas_prices_records

,start_date,end_date,product,average_retail_price,state,date_range_id
46364,2020-12-27,2021-01-02,GASOLINA COMUM,5.088,AC,1
46365,2020-12-27,2021-01-02,GASOLINA COMUM,4.673,AL,1
46366,2020-12-27,2021-01-02,GASOLINA COMUM,3.830,AP,1
46367,2020-12-27,2021-01-02,GASOLINA COMUM,4.305,AM,1
46368,2020-12-27,2021-01-02,GASOLINA COMUM,4.453,BA,1
...,...,...,...,...,...,...
47754,2021-04-25,2021-05-01,GASOLINA COMUM,5.364,RR,18
47755,2021-04-25,2021-05-01,GASOLINA COMUM,5.051,SC,18
47756,2021-04-25,2021-05-01,GASOLINA COMUM,5.265,SP,18
47757,2021-04-25,2021-05-01,GASOLINA COMUM,5.586,SE,18


In [101]:
# Adiciona 'date_range_id' a 'delivery_orders' usando a função auxiliar:
delivery_orders['date_range_id'] = delivery_orders.apply(lambda x: find_date_range(x, gas_prices_records), axis=1)

delivery_orders

,delivery_order_id,order_date,paused_time_minutes,speed_time_minutes,transit_time_minutes,delivery_distance_meters,city,state,date_range_id
17,68412721,2021-01-01,10.22,11.05,83.30,5714.0,SÃO PAULO,SP,1
18,68413340,2021-01-01,3.10,12.67,49.78,5199.0,SÃO PAULO,SP,1
19,68414018,2021-01-01,3.03,13.28,11.05,3746.0,SÃO PAULO,SP,1
21,68414512,2021-01-01,39.75,39.85,23.82,5514.0,SÃO PAULO,SP,1
26,68415344,2021-01-01,28.83,29.38,53.08,5385.0,SÃO PAULO,SP,1
...,...,...,...,...,...,...,...,...,...
389181,93139802,2021-04-30,0.63,4.05,21.80,1489.0,SÃO PAULO,SP,18
389182,93139805,2021-04-30,3.52,9.68,5.83,475.0,SÃO PAULO,SP,18
389184,93139809,2021-04-30,3.48,9.62,20.67,3735.0,SÃO PAULO,SP,18
389185,93139813,2021-04-30,6.38,18.13,17.55,3931.0,SÃO PAULO,SP,18


In [102]:
# Reseta os índices dos dois dataframes antes do merge:
delivery_orders = delivery_orders.reset_index(drop=True)
gas_prices_records = gas_prices_records.reset_index(drop=True)

# Une dataframes 'gas_prices_records' e 'delivery_orders' por meio de 'state' e 'date_range_id':
merged_df = pd.merge(
    delivery_orders,
    gas_prices_records[['state', 'date_range_id', 'product', 'average_retail_price']],
    on=['state', 'date_range_id'],
    how='left'
)

merged_df

,delivery_order_id,order_date,paused_time_minutes,speed_time_minutes,transit_time_minutes,delivery_distance_meters,city,state,date_range_id,product,average_retail_price
0,68412721,2021-01-01,10.22,11.05,83.30,5714.0,SÃO PAULO,SP,1,GASOLINA COMUM,4.235
1,68413340,2021-01-01,3.10,12.67,49.78,5199.0,SÃO PAULO,SP,1,GASOLINA COMUM,4.235
2,68414018,2021-01-01,3.03,13.28,11.05,3746.0,SÃO PAULO,SP,1,GASOLINA COMUM,4.235
3,68414512,2021-01-01,39.75,39.85,23.82,5514.0,SÃO PAULO,SP,1,GASOLINA COMUM,4.235
4,68415344,2021-01-01,28.83,29.38,53.08,5385.0,SÃO PAULO,SP,1,GASOLINA COMUM,4.235
...,...,...,...,...,...,...,...,...,...,...,...
263925,93139802,2021-04-30,0.63,4.05,21.80,1489.0,SÃO PAULO,SP,18,GASOLINA COMUM,5.265
263926,93139805,2021-04-30,3.52,9.68,5.83,475.0,SÃO PAULO,SP,18,GASOLINA COMUM,5.265
263927,93139809,2021-04-30,3.48,9.62,20.67,3735.0,SÃO PAULO,SP,18,GASOLINA COMUM,5.265
263928,93139813,2021-04-30,6.38,18.13,17.55,3931.0,SÃO PAULO,SP,18,GASOLINA COMUM,5.265


Antes de finalizar, porém, o tratamento dos dados, aproveitamos para validar as lacunas no dataframe mesclado, para nos assegurarmos de que ele ainda cumpre com os requisitos mínimos do projeto e que não tenha valores vazios e negativos indesejados que impeçam o funcionamento esperado das etapas de análise.

In [103]:
# Confere lacunas em cada coluna de merged_df:
valores_nulos = merged_df.isnull().sum()
print(f"Valores nulos em merged_df:\n {valores_nulos}\n")

# Confere total de valores não nulos em cada coluna de merged_df:
valores_nao_nulos = merged_df.notnull().sum()
print(f"Valores não nulos em merged_df:\n {valores_nao_nulos}")

Valores nulos em merged_df:
 delivery_order_id               0
order_date                      0
paused_time_minutes         40116
speed_time_minutes          10461
transit_time_minutes         3023
delivery_distance_meters       25
city                            0
state                           0
date_range_id                   0
product                         0
average_retail_price            0
dtype: int64

Valores não nulos em merged_df:
 delivery_order_id           263930
order_date                  263930
paused_time_minutes         223814
speed_time_minutes          253469
transit_time_minutes        260907
delivery_distance_meters    263905
city                        263930
state                       263930
date_range_id               263930
product                     263930
average_retail_price        263930
dtype: int64


In [104]:
# Filtra final_df para remover as lacunas nas colunas especificadas:
final_df = merged_df.dropna(subset=['paused_time_minutes', 
                                             'speed_time_minutes', 
                                             'transit_time_minutes', 
                                             'delivery_distance_meters'])

# Filtra final_df para manter somente os campos onde as colunas especificadas forem positivas:
final_df = final_df[
    (final_df['paused_time_minutes'] >= 0) &
    (final_df['speed_time_minutes'] >= 0) &
    (final_df['transit_time_minutes'] >= 0) &
    (final_df['delivery_distance_meters'] >= 0) &
    (final_df['average_retail_price'] >= 0)
]

# Exibe o dataframe final e o exporta para csv:
final_df.to_csv('final_df.csv', index=False)
final_df

,delivery_order_id,order_date,paused_time_minutes,speed_time_minutes,transit_time_minutes,delivery_distance_meters,city,state,date_range_id,product,average_retail_price
0,68412721,2021-01-01,10.22,11.05,83.30,5714.0,SÃO PAULO,SP,1,GASOLINA COMUM,4.235
1,68413340,2021-01-01,3.10,12.67,49.78,5199.0,SÃO PAULO,SP,1,GASOLINA COMUM,4.235
2,68414018,2021-01-01,3.03,13.28,11.05,3746.0,SÃO PAULO,SP,1,GASOLINA COMUM,4.235
3,68414512,2021-01-01,39.75,39.85,23.82,5514.0,SÃO PAULO,SP,1,GASOLINA COMUM,4.235
4,68415344,2021-01-01,28.83,29.38,53.08,5385.0,SÃO PAULO,SP,1,GASOLINA COMUM,4.235
...,...,...,...,...,...,...,...,...,...,...,...
263923,93139799,2021-04-30,1.87,4.18,13.22,2911.0,CURITIBA,PR,18,GASOLINA COMUM,5.311
263925,93139802,2021-04-30,0.63,4.05,21.80,1489.0,SÃO PAULO,SP,18,GASOLINA COMUM,5.265
263926,93139805,2021-04-30,3.52,9.68,5.83,475.0,SÃO PAULO,SP,18,GASOLINA COMUM,5.265
263927,93139809,2021-04-30,3.48,9.62,20.67,3735.0,SÃO PAULO,SP,18,GASOLINA COMUM,5.265
